In [2]:
import os
import time
import random
import shutil
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
from torchvision import datasets, models, transforms
from torch.utils.data import TensorDataset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from PIL import Image, ImageOps

In [7]:
!pip3 install python-Levenshtein

  Obtaining dependency information for python-Levenshtein from https://files.pythonhosted.org/packages/27/89/c45dbdbd479453cfc8c4c1271c9f67fd607deaf84880e31c67b682980456/python_Levenshtein-0.23.0-py3-none-any.whl.metadata
  Obtaining dependency information for Levenshtein==0.23.0 from https://files.pythonhosted.org/packages/12/06/c4a6bbc804d027ccc2b0229adb7ed733b63db34bb18d261aeae6c2fb2ea4/Levenshtein-0.23.0-cp311-cp311-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for rapidfuzz<4.0.0,>=3.1.0 from https://files.pythonhosted.org/packages/5d/47/309addcb9e8d818a8456956ffa9f4db57be11187515b64d24ca0d500e3a4/rapidfuzz-3.5.2-cp311-cp311-macosx_10_9_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 17.9 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
from Levenshtein import distance as levenshtein_distance

def wer(reference, hypothesis):
    # Split the sentences into words
    ref = reference.split()
    hyp = hypothesis.split()

    # Create a matrix of size (len(ref)+1) x (len(hyp)+1)
    d = np.zeros((len(ref) + 1) * (len(hyp) + 1), dtype=np.uint8)
    d = d.reshape((len(ref) + 1, len(hyp) + 1))

    # Initialize the first row and column
    for i in range(len(ref) + 1):
        d[i][0] = i
    for j in range(len(hyp) + 1):
        d[0][j] = j

    # Populate the matrix
    for i in range(1, len(ref) + 1):
        for j in range(1, len(hyp) + 1):
            if ref[i - 1] == hyp[j - 1]:
                cost = 0
            else:
                cost = 1
            d[i][j] = min(d[i - 1][j] + 1,      # deletion
                          d[i][j - 1] + 1,      # insertion
                          d[i - 1][j - 1] + cost)  # substitution

    # Compute WER
    wer_value = float(d[len(ref)][len(hyp)]) / len(ref)

    return wer_value

In [9]:
def ground_truth(file_name):
  #ground truth = [date,name,total]
  f = open(file_name, 'r')
  content = f.read()

  #print(content)
  content_split = content.split("\n")

  #print(content_split)
  ground_label = [0,0,0]

  for i in content_split:
    if "date" in i:
      a = i.split(":")
      a[1] = a[1].replace(",","")
      a[1] = a[1].replace('"',"")
      #print("date ", a[1])
      ground_label[0] = a[1]

    elif "company" in i:
      a = i.split(":")
      a[1] = a[1].replace(",","")
      a[1] = a[1].replace('"',"")
      #print("name ", a[1])
      ground_label[1] = a[1]

    elif "total" in i:
      a = i.split(":")
      a[1] = a[1].replace(",","")
      a[1] = a[1].replace('"',"")
      #print("total ", a[1])
      ground_label[2] = a[1]

  return(ground_label)

In [10]:
# Example dataset containing unique characters
unique_characters = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!@#$%^&*()_+-=[]{};:,.<>?/\\| "

# Calculate the number of unique classes
NUM_CLASSES = len(unique_characters)
print("Number of classes:", NUM_CLASSES)

Number of classes: 91


In [11]:
num_classes = NUM_CLASSES
class OCRModel(nn.Module):
    def __init__(self, num_classes):
        super(OCRModel, self).__init__()
        # CNN layers
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),  # Output: 32x128 -> 32x128 (after pooling: 16x64)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),  # Reduces dimensions by half
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),  # Output: 16x64 -> 16x64 (after pooling: 8x32)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),  # Further reduces dimensions by half
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1), # Additional layer, Output: 8x32 -> 8x32 (after pooling: 4x16)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        # RNN layers
        # Input size calculation: Final CNN output size is [batch_size, 128 channels, 4 height, 16 width]
        # Flattening height into channels -> [batch_size, 512, 16]
        self.rnn = nn.LSTM(input_size=512, hidden_size=256, num_layers=2, batch_first=True, bidirectional=True)
        # Fully connected layer
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        # CNN
        x = self.cnn(x)
        print("Shape after CNN:", x.shape)  # Check shape after CNN layers

        # Flattening the height and channel dimensions
        batch_size, channels, height, width = x.size()
        x = x.view(batch_size, channels * height, width)  # New shape: [batch_size, channels * height, width]
        print("Shape after flattening:", x.shape)

        x = x.permute(0, 2, 1)  # Permute to make width as sequence length
        # RNN
        x, _ = self.rnn(x)
        # Fully connected layer
        x = self.fc(x)
        return x


In [12]:
image_folder = '/content/drive/MyDrive/Colab Notebooks/test_img'
label_folder = '/content/drive/MyDrive/Colab Notebooks/entities'
csv_file = '/content/drive/MyDrive/Colab Notebooks/output.csv'

# data = []

# # Iterate through each image in the image folder
# for image_file in os.listdir(image_folder):
#     if image_file.endswith(('.png', '.jpg', '.jpeg')):  # Add other image formats if needed
#         image_path = os.path.join(image_folder, image_file)

#         # Construct the corresponding label file path
#         label_file = os.path.splitext(image_file)[0] + '.txt'  # Assuming text files have .txt extension
#         label_path = os.path.join(label_folder, label_file)

#         # Read the label content
#         if os.path.exists(label_path):
#             with open(label_path, 'r') as file:
#                 label = ground_truth(label_path)
#                 data.append([image_path, label])

# # Create a DataFrame and save as CSV
# df = pd.DataFrame(data, columns=['image_path', 'label'])
# df.to_csv(csv_file, index=False)

In [ ]:
image_folder = ''

In [ ]:
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import gc

# Custom dataset class
class ReceiptsDataset(Dataset):
    def __init__(self, csv_file, transform=None, fixed_height=128, max_width=4096):
        self.dataframe = pd.read_csv(csv_file)
        self.transform = transform
        self.fixed_height = fixed_height
        self.max_width = max_width

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx, 0]
        image = Image.open(img_name).convert('L')
        image = self.resize_and_pad(image)
        label = self.dataframe.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)

        return image, label

    def resize_and_pad(self, image):
        aspect_ratio = image.width / image.height
        new_width = int(self.fixed_height * aspect_ratio)
        resized_image = image.resize((new_width, self.fixed_height), Image.LANCZOS)

        if new_width < self.max_width:
            padding_left = (self.max_width - new_width) // 2
            padding_right = self.max_width - new_width - padding_left
            padding_top = padding_bottom = 0
            resized_image = ImageOps.expand(resized_image, border=(padding_left, padding_top, padding_right, padding_bottom), fill=0)

        return resized_image

# Transformations
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Load dataset
train_dataset = ReceiptsDataset(csv_file='/content/drive/MyDrive/Colab Notebooks/output.csv', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Model, loss, and optimizer
num_classes = 100  # Set this to your number of classes (characters)
model = OCRModel(num_classes)
ctc_loss = nn.CTCLoss(blank=0)  # Assuming 0 is the index for the blank character
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
unique_chars = set()
for _, label in train_dataset:
    unique_chars.update(label)

# Add a special character for CTC blank token, if it's not already included
unique_chars.add('')

# Create char_to_int mapping
char_to_int = {char: i for i, char in enumerate(sorted(unique_chars))}
# Int to char
#int_to_char = {i: char for char, i in char_to_int.items()}

num_epochs = 10  # Set the number of epochs
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    total_loss = 0

    for batch_idx, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)  # outputs shape after model: [batch_size, channels * height, width]

        # Correctly permute and reshape outputs for CTC
        outputs = outputs.permute(2, 0, 1)  # now shape: [width, batch_size, channels * height]
        outputs = outputs.reshape(512, 16, num_classes)

        # Flatten all labels into a single list and calculate label_lengths
        flat_labels = []
        label_lengths = []
        for label in labels:
            char_indices = [char_to_int[char] for char in label]  # Convert each char to its index
            flat_labels.extend(char_indices)
            label_lengths.append(len(char_indices))

        # Convert lists to tensors
        flat_labels = torch.IntTensor(flat_labels)
        label_lengths = torch.IntTensor(label_lengths)
        input_lengths = torch.full(size=(32,), fill_value=620, dtype=torch.int32)

        # Check if label_lengths size matches batch size
        if label_lengths.size(0) != images.size(0):
            raise ValueError(f"Size mismatch: label_lengths ({label_lengths.size(0)}) vs batch_size ({images.size(0)})")

        # Calculate the input lengths for CTC Loss
        input_lengths = torch.full(size=(outputs.size(1),), fill_value=outputs.size(0), dtype=torch.int32)

        # Calculate CTC Loss
        loss = ctc_loss(outputs.log_softmax(2), flat_labels, input_lengths, label_lengths)
        total_loss += loss.item()

        # Backward pass: Compute gradient of the loss with respect to model parameters
        loss.backward()

        # Perform a single optimization step (parameter update)
        optimizer.step()
        del images, outputs, flat_labels, label_lengths, input_lengths, loss
        gc.collect()

        if batch_idx % 10 == 0:  # Print loss every 10 batches
            print(f"Epoch: {epoch+1}, Batch: {batch_idx+1}, Loss: {loss.item()}")

    print(f"Epoch: {epoch+1}, Average Loss: {total_loss / len(train_loader)}")

Shape after CNN: torch.Size([16, 128, 16, 512])
Shape after flattening: torch.Size([16, 2048, 512])


In [ ]:
    # Function to convert model output to text
def decode_ctc_output(output, char_map):
    # Applying softmax to get probabilities
    probs = torch.softmax(output, 2)
    probs = probs.permute(1, 0, 2)  # Reshape to batch first
    max_probs, indices = torch.max(probs, 2)

    # Convert indices to characters
    texts = []
    for i in range(indices.size(0)):
        text = ''.join([char_map[index] for index in indices[i] if index != 0])  # 0 is the CTC blank character
        texts.append(text)

    return texts

# Function to process an image and predict text
def predict_text(model, image_path, char_map):
    # Load and transform the image
    image = Image.open(image_path).convert('L')
    transform = transforms.Compose([transforms.ToTensor()])
    image = transform(image)
    image = image.unsqueeze(0)  # Add batch dimension

    # Forward pass
    model.eval()
    with torch.no_grad():
        output = model(image)

    # Decode the output to text
    predicted_text = decode_ctc_output(output, char_map)

    return predicted_text

int_to_char = {i: char for char, i in char_to_int.items()}

# Example usage
image_path = '/content/drive/MyDrive/Colab Notebooks/test_img/X00016469670/jpg'
predicted_text = predict_text(model, image_path, int_to_char)
print(f"Predicted text: {predicted_text[0]}")

In [ ]:
def test_model_on_images(model, image_folder, char_map, label_folder):
    for image_file in os.listdir(image_folder):
        if image_file.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(image_folder, image_file)
            predicted_text = predict_text(model, image_path, char_map)[0]

            # Construct the corresponding label file path
            label_file = os.path.splitext(image_file)[0] + '.txt'
            label_path = os.path.join(label_folder, label_file)

            # Read the ground truth label
            if os.path.exists(label_path):
                ground_truth_label = ' '.join(ground_truth(label_path))  # Joining the ground truth elements to form a sentence

                # Calculate WER
                error_rate = wer(ground_truth_label, predicted_text)

                # Print results
                print(f"Image: {image_file}, Predicted text: {predicted_text}, WER: {error_rate:.2f}")

# Test the model
model = OCRModel
test_image_folder = '/content/drive/MyDrive/test/img'
test_model_on_images(model, test_image_folder, int_to_char)